<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Получение-и-подготовка-данных" data-toc-modified-id="Получение-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Получение и подготовка данных</a></span></li><li><span><a href="#Дашборд-в-Tableau-Public" data-toc-modified-id="Дашборд-в-Tableau-Public-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Дашборд в Tableau Public</a></span></li><li><span><a href="#Презентация" data-toc-modified-id="Презентация-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Презентация</a></span></li></ul></div>

## Описание проекта

Данный проект посвящён подготовке и созданию дашборда на Tableau Public. Дашборд поможет проанализировать итоги опроса и определить текущий уровень потребительской лояльности (NPS - Net Promoter Score) клиентов большой телекоммуникационной компании.

**Основные задачи:**

Создать дашборд, который поможет ответить на вопросы заказчика:
- Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?


**План работы:**

- скачать файл с базой данных
- создать коннекцию к базе
- выполнить sql-запрос
- проверить данные
- выгрузить данные из таблицы в csv-файл 
- создать дашборд в Tableau Public
- прикрепить ссылки на готовый дашборд и презентацию

**Описание данных:**

Дашборд основан на информации из базы данных компании.

## Получение и подготовка данных

Импортируем библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Создаём коннекцию к базе и sql-запрос:

In [2]:
path_to_db = '/datasets/telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

query = """
SELECT u.user_id AS user_id,
       u.lt_day AS lt_day,
       CASE                                        -- создаём новый столбец с условием
           WHEN lt_day <= 365 THEN 'yes'           -- если длина "жизни" клиента в сервисе не больше года, 
           ELSE 'no'                               -- то он для нас новый клиент, иначе - нет
       END AS is_new,
       u.age AS age,
       CASE 
           WHEN gender_segment == 1 THEN 'female'  -- преобразуем значения 1.0 и 0.0 в текстовый вид
           WHEN gender_segment == 0 THEN 'male'    -- второе условие нужно, чтобы не сделать пропуски значением male
       END AS gender_segment,
       u.os_name AS os_name,
       u.cpe_type_name AS cpe_type_name,
       l.country AS country,
       l.city AS city,
       age_s.title AS age_segment,
       tr_s.title AS traffic_segment,
       lt_s.title AS lifetime_segment,
       u.nps_score AS nps_score,
       CASE                                        -- создаём новый столбец с категориями групп оценок
           WHEN nps_score > 8 THEN 'promoters'     -- 9 и 10 - такие оценки ставят сторонники 
           WHEN nps_score < 7 THEN 'detractors'    -- от 0 до 6 - такие оценки ставят критики
           ELSE 'passives'                         -- остальные оценки (7 и 8) ставят нейтралы
       END AS nps_group
       
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS age_s ON u.age_gr_id = age_s.age_gr_id
LEFT JOIN traffic_segment AS tr_s ON u.tr_gr_id = tr_s.tr_gr_id
LEFT JOIN lifetime_segment AS lt_s ON u.lt_gr_id = lt_s.lt_gr_id;
"""

df = pd.read_sql(query, engine)

Проверяем данные собственной функцией:

In [3]:
def my_check_function (dataset):
    '''Функция для ознакомления и проверки даных'''
    print('Как выглядят случайные 5 строк:')
    display(dataset.sample(5, random_state=42))
    print()
    print('Общая информация о данных, наименования столбцов, типы данных:')
    print(dataset.info())
    print()
    print('Есть ли пропуски, сколько их:')
    print(dataset.isna().sum())
    print('Доля пропущенных значений:')
    print(dataset.isna().mean())
    print()
    print('Есть ли явные дубликаты, сколько их:')
    print(dataset.duplicated().sum())
    print('Доля явных дубликатов:')
    print(dataset.duplicated().mean())
    print()
    print('Подробное описание данных:')
    display(dataset.describe())

In [4]:
my_check_function(df)

Как выглядят случайные 5 строк:


,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
375810,TFV382,596,no,28.0,female,IOS,SMARTPHONE,Россия,Екатеринбург,03 25-34,05 5-10,06 13-24,2,detractors
20818,B2QEQO,1492,no,45.0,male,ANDROID,SMARTPHONE,Россия,Екатеринбург,05 45-54,05 5-10,08 36+,7,passives
14854,ARHTR2,472,no,41.0,female,ANDROID,SMARTPHONE,Россия,Томск,04 35-44,05 5-10,06 13-24,8,passives
413807,VEHIHV,1842,no,31.0,male,ANDROID,SMARTPHONE,Россия,НижнийНовгород,03 25-34,07 15-20,08 36+,3,detractors
417174,VKTHQA,630,no,44.0,female,ANDROID,SMARTPHONE,Россия,Новокузнецк,04 35-44,04 1-5,06 13-24,10,promoters



Общая информация о данных, наименования столбцов, типы данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


Как мы видим, данные в основном в порядке. Есть некоторые пропуски с информацией о поле и возрасте, но переиначивать их нет смысла. При этом есть более странная вещь - отрицательные дни "жизни" клиента (lt_day - значение min - отрицательное число). Посмотрим на такие "отрицательные" строки поближе.

In [5]:
df.loc[df['lt_day'] < 0]

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
7666,AEAC2R,-8,yes,NaN,None,ANDROID,SMARTPHONE,Россия,Томск,08 n/a,07 15-20,08 36+,1,detractors
53997,CS0HF8,-2,yes,NaN,None,ANDROID,SMARTPHONE,Россия,Ижевск,08 n/a,08 20-25,08 36+,10,promoters
71577,DORPT2,-4,yes,NaN,None,ANDROID,SMARTPHONE,Россия,Москва,08 n/a,08 20-25,08 36+,4,detractors
100152,F5O3CG,-21,yes,NaN,None,ANDROID,SMARTPHONE,Россия,Чита,08 n/a,11 35-40,08 36+,7,passives
228087,LSE939,-13,yes,NaN,None,ANDROID,SMARTPHONE,Россия,Краснодар,08 n/a,07 15-20,08 36+,1,detractors
325212,QTT7IR,-13,yes,NaN,None,IOS,SMARTPHONE,Россия,Москва,08 n/a,06 10-15,08 36+,7,passives
347784,RZUS6H,-6,yes,NaN,None,ANDROID,SMARTPHONE,Россия,Красноярск,08 n/a,10 30-35,08 36+,3,detractors
385897,TYWQW4,-12,yes,NaN,None,IOS,SMARTPHONE,Россия,Новосибирск,08 n/a,04 1-5,08 36+,5,detractors
401226,URKAFI,-11,yes,NaN,None,ANDROID,SMARTPHONE,Россия,НижнийНовгород,08 n/a,06 10-15,08 36+,8,passives
407160,V2GNK2,-13,yes,NaN,None,IOS,SMARTPHONE,Россия,РостовнаДону,08 n/a,07 15-20,08 36+,5,detractors


Как мы видим, у нас есть 13 таких строк, где количество дней "жизни" клиента в сервисе выражено отрицательным числом, в остальных строках данные разнообразны: здесь и разные города, и типы операционных систем, и сегменты трафика, и разные оценки.
Что общего: неизвестен возраст, пол, все пользуются сервисом через смартфон, и сегмент "жизни" пользователя в месяцах явно противоречит длине такой "жизни" в днях. То есть по одной информации, это люди, которые пользуются более трёх лет (lifetime_segment 36+), а по другой (lt_day) они даже не начинали пользоваться (отрицательное число).

Возможные причины появления отрицательных чисел:
- техническая ошибка при заполнении, переносе данных
- ошибка, связанная с лимитом заполнения (настоящее число выходит за максимальное значение)
- если сервис платный, то так отражается задолженность или факт отключения в текущем месяце (все отрицательные числа в пределах 30 дней)

У нас в данных 502493 строки. Думаю, эти подозрительные 13 строк можно убрать из исследования, и это никоим образом не исказит результаты работы и не помешает ответить на вопросы заказчика.

1. Поскольку в ТЗ отмечено, что преобразовывать данные можно только с помощью SQL-запросов, вернёмся к этому этапу и внесём изменения в запросе (новый запрос ниже). 

2. Ещё можно для удобства переименовать значения в столбце is_new: ранее я писала, если новый клиент - yes, если старый - no. Заменю это на формулировку: если новый клиент - new, если старый - old, так будет удобнее смотреть на графики в дальнейшем.

3. Для удобства восприятия и анализа в столбцах с составным наименованием сегментов оставляем только диапазон значений, применяя фунцию substr()


In [6]:
path_to_db = '/datasets/telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

query = """
SELECT u.user_id AS user_id,
       CASE
           WHEN lt_day >= 0 THEN lt_day            -- добавляем новое условие, отсекая подозрительные строки с отриц.днями
       END AS lt_day,
       CASE                                        -- создаём новый столбец с условием
           WHEN lt_day <= 365 THEN 'new'           -- если длина "жизни" клиента в сервисе не больше года, 
           ELSE 'old'                              -- то он для нас новый клиент, иначе - старый
       END AS is_new,
       u.age AS age,
       CASE 
           WHEN gender_segment == 1 THEN 'female'  -- преобразуем значения 1.0 и 0.0 в текстовый вид
           WHEN gender_segment == 0 THEN 'male'    -- второе условие нужно, чтобы не сделать пропуски значением male
       END AS gender_segment,
       u.os_name AS os_name,
       u.cpe_type_name AS cpe_type_name,
       l.country AS country,
       l.city AS city,
       substr(age_s.title, 4) AS age_segment,      -- для удобства восприятия и анализа в столбцах с 
       substr(tr_s.title, 4) AS traffic_segment,   -- составным наименованием сегментов оставляем только диапазон значений,
       substr(lt_s.title, 4) AS lifetime_segment,  -- а номер группы, который указан в начале, отсеиваем 
       u.nps_score AS nps_score,
       CASE                                        -- создаём новый столбец с категориями групп оценок
           WHEN nps_score > 8 THEN 'promoters'     -- 9 и 10 - такие оценки ставят сторонники 
           WHEN nps_score < 7 THEN 'detractors'    -- от 0 до 6 - такие оценки ставят критики
           ELSE 'passives'                         -- остальные оценки (7 и 8) ставят нейтралы
       END AS nps_group
       
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS age_s ON u.age_gr_id = age_s.age_gr_id
LEFT JOIN traffic_segment AS tr_s ON u.tr_gr_id = tr_s.tr_gr_id
LEFT JOIN lifetime_segment AS lt_s ON u.lt_gr_id = lt_s.lt_gr_id;
"""

df_telecom = pd.read_sql(query, engine)

Проверяем, как теперь выглядит наш датафрейм:

In [7]:
df_telecom.head() 

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320.0,old,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344.0,old,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467.0,old,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190.0,old,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163.0,old,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


In [8]:
len(df.loc[df['lt_day'] < 0]) / len(df) * 100

0.002587100715830867

Из данных исключены 13 строк, что составляет 0.0026%

Сохраняем данные в файл csv, будем использовать его в Tableau Public:

In [9]:
df_telecom.to_csv('df_telecom.csv', index=False) 

## Дашборд в Tableau Public

Tableau Public:

https://public.tableau.com/app/profile/natalya.malakhova/viz/telecom_project_16634177186890/Dashboard1

## Презентация

Презентация: https://disk.yandex.ru/i/-Ldyg9hjsYeNiA

Спасибо за внимание!